### **Do people with different ideologies speak differently?**
ADA Project Milestone P2

# Mouting the Google Drive

It is possible to mount your Google Drive to Colab if you need additional storage or if you need to use files from it. To do that run (click on play button or use keyboard shortcut 'Command/Ctrl+Enter') the following code cell:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

 

1.   After running the cell, URL will appear.

2.   Following this URL, you will be redirected to the page where you need to choose Google Drive account to mount to.

3.   You will further be asked to give Google Drive Stream a permission to access the chosen Google account

4.   After granting the access, authorization code will be given to you

5.   Copy the authorization code into the dedicated textbox in Colab under '*Enter your authorization code:*' writing

After copying the authorization code, you should get the message saying '*Mounted at /content/gdrive*'

Path to the files from the mounted Drive will then be '/content/drive/MyDrive/'. By opening the Files tab (left sidebar, folder icon) you should also be able to see the accessible files.

# Cleaning of the data

In [ ]:
!pip install pandas==1.0.5

In [ ]:
# Imports
import bz2
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

## Load/filter/merge for all years quotebank and wikidata samples<br>
Definition of functions used during data cleaning and preprocessing

In [ ]:
# Load and filter wikidata for our purpose
def load_filter_wikidata_df(path_to_file):
  # load from file
  columns = ["id", "gender", "occupation", "party"]
  df_wikidata = pd.read_parquet(path_to_file, columns=columns)

  # Filter
  # Remove rows without a party
  df_wikidata_parties = df_wikidata.dropna(subset=['party'])

  # Select only rows with either republican or democrats party
  QID_republicans = "Q29468"
  QID_democrats = "Q29552"
  df_wikidata_filtered = df_wikidata_parties[df_wikidata_parties.apply(lambda x: (QID_republicans in x['party']) or (QID_democrats in x['party']) , axis=1)]

  return df_wikidata_filtered

In [ ]:
# Perform all operations needed on quotebank dataset (load/filter/merge with wikidata/store)
def handle_quotebank_df(input_file, output_file, chunk_size, df_wikidata, n_chunks=0):
  curr_chunk = 0
  chunk_list = []

  # read input file by chunks (as the whole file can't fit into memory)
  reader = pd.read_json(input_file, lines=True, compression='bz2', chunksize=chunk_size)
  for chunk in reader:
    #if curr_chunk == n_chunks:
     # break
    curr_chunk += 1
    # append only when the speaker is knows (the best % is not from "None" speaker)
    filtered_chunk = chunk[chunk['speaker'] != 'None' ][['quoteID', 'quotation','speaker', 'qids', 'probas']]
    
    # apply filter to single chunk
    filtered_chunk = filter_quotebank_df(filtered_chunk)

    # merge single chunk with wikidata in order to reduce even further the dataset and allow RAM to store it
    filtered_merged_chunk = merge_quotebank_wikidata_df(filtered_chunk, df_wikidata)
    
    # append to output file a single chunk
    filtered_merged_chunk.to_csv(output_file, mode='a', compression='bz2')
    

In [ ]:
# Filter unuset entries in wikidata
def filter_quotebank_df(df):
  # remove the data with not unique qid speaker because we are not sure who is the speaker: speakers with same name but different qids
  df_filtered = df[df.apply(lambda x: len(x['qids']) == 1, axis=1)]

  # now we don't have anymore list of quids (only 1 quid per entry possible), so remove list and store only the single value
  df_filtered['qids'] = df_filtered['qids'].apply(lambda x: x[0])

  return df_filtered

In [ ]:
# Merge quotebank and wikidata entries on QID
def merge_quotebank_wikidata_df(df_quotebank, df_wikidata):
  #merge quotebank data with wikidata 
  df_merged = df_quotebank.merge(right=df_wikidata, how='inner', left_on='qids', right_on='id')

  #drop the id column because we already have the qid
  df_merged = df_merged.drop(labels='id', axis=1)
  
  return df_merged

In [ ]:
# Append to output file a single chunk
def store_chunk_df(path_to_file, df):
  # Dump the single chunk to csv, appending it to previously written chunks
  filtered_merged_chunk.to_csv(output_file, mode='a', compression='bz2')

Actual data cleaning and preprocessing is done here. The final dataframe for each year is saved in an additional .bz2 file.<br>
*(Note that it takes around 5 hours to run the following cell)*

In [ ]:
chunk_size = 100000
# n_chunks = 10

path_to_parquet = '/content/drive/MyDrive/Project datasets/speaker_attributes.parquet'
df_wikidata = load_filter_wikidata_df(path_to_parquet)

input_file = '/content/drive/MyDrive/Quotebank/quotes-2020.json.bz2'
output_file = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2020-repub-dem.json.bz2'
handle_quotebank_df(input_file, output_file, chunk_size, df_wikidata)

input_file = '/content/drive/MyDrive/Quotebank/quotes-2019.json.bz2'
output_file = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2019-repub-dem.json.bz2'
handle_quotebank_df(input_file, output_file, chunk_size, df_wikidata)

input_file = '/content/drive/MyDrive/Quotebank/quotes-2018.json.bz2'
output_file = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2018-repub-dem.json.bz2'
handle_quotebank_df(input_file, output_file, chunk_size, df_wikidata)

input_file = '/content/drive/MyDrive/Quotebank/quotes-2017.json.bz2'
output_file = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2017-repub-dem.json.bz2'
handle_quotebank_df(input_file, output_file, chunk_size, df_wikidata)

input_file = '/content/drive/MyDrive/Quotebank/quotes-2016.json.bz2'
output_file = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2016-repub-dem.json.bz2'
handle_quotebank_df(input_file, output_file, chunk_size, df_wikidata)

input_file = '/content/drive/MyDrive/Quotebank/quotes-2015.json.bz2'
output_file = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2015-repub-dem.json.bz2'
handle_quotebank_df(input_file, output_file, chunk_size, df_wikidata)


## Load and merge preprocessed data for all years (2015-2020)

In [ ]:
file_path_2020 = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2020-repub-dem.json.bz2'
file_path_2019 = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2019-repub-dem.json.bz2'
file_path_2018 = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2018-repub-dem.json.bz2'
file_path_2017 = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2017-repub-dem.json.bz2'
file_path_2016 = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2016-repub-dem.json.bz2'
file_path_2015 = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2015-repub-dem.json.bz2'

def load_and_merge():
  df = pd.read_csv(file_path_2020, compression='bz2')
  df_1 = pd.read_csv(file_path_2019, compression='bz2')
  df = pd.concat([df, df_1], ignore_index=True)
  print("Done")
  df_1 = pd.read_csv(file_path_2018, compression='bz2')
  df = pd.concat([df, df_1], ignore_index=True)
  print("Done")
  df_1 = pd.read_csv(file_path_2017, compression='bz2')
  df = pd.concat([df, df_1], ignore_index=True)
  print("Done")
  df_1 = pd.read_csv(file_path_2016, compression='bz2')
  df = pd.concat([df, df_1], ignore_index=True)
  print("Done")
  df_1 = pd.read_csv(file_path_2015, compression='bz2')
  df = pd.concat([df, df_1], ignore_index=True)
  print("Done")

  return df

In [ ]:
df = load_and_merge()
df

### OLD CODE

In [ ]:
chunk_size = 100000
n_chunks = 10
input_file = '/content/drive/MyDrive/Quotebank/quotes-2020.json.bz2'

def load_quotebank_df(path_to_file, chunk_size, n_chunks=0):
  curr_chunk = 0
  chunk_list = []
  reader = pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunk_size)
  for chunk in reader:
    #if curr_chunk == n_chunks:
     # break
    curr_chunk += 1
    # append only when the speaker is knows (the best % is not from "None" speaker)
    chunk_list.append(chunk[chunk['speaker'] != 'None' ][['quoteID', 'quotation','speaker', 'qids', 'probas']])
  df = pd.concat(chunk_list)
  print(curr_chunk)
  return df

df = load_quotebank_df(input_file, chunk_size, n_chunks)
df.head()

In [ ]:
len(df)

check if there are duplicate quote IDs

In [ ]:
(df['quoteID'].value_counts() > 1).any()

check some conditions

remove the data with not unique qid speaker because we are not sure who is the speaker: speakers with same name but different qids




In [ ]:
qid_array = df['qids'].values
# number of qids per speaker
n_qids = [len(list) for list in qid_array ]
filtered = filter(lambda n: n > 1, n_qids)
len(list(filtered))

In [ ]:
df_filtered = df[df.apply(lambda x: len(x['qids']) == 1, axis=1)]

In [ ]:
len(df_filtered)

In [ ]:
# Since we removed the persons with more than on qid we can remove the list and leave only the value inside the list qids
df_filtered['qids'] = df_filtered['qids'].apply(lambda x: x[0])

In [ ]:
df_filtered.head()

# Merge some attributes of the speakers from Wikidata 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
path_to_parquet = '/content/drive/MyDrive/Project datasets/speaker_attributes.parquet'
columns = ["id", "gender", "occupation", "party"]
df_wikidata = pd.read_parquet(path_to_parquet, columns=columns)


In [ ]:
df_wikidata

filter none partys

In [ ]:
df_wikidata_parties = df_wikidata.dropna(subset=['party'])
df_wikidata_parties

Check if we have some rows with more one qid for the party

In [ ]:
qid_array_party = df_wikidata_parties['party'].values
# number of qids per speaker
n_qids_party = [len(list) for list in qid_array_party ]
filtered = filter(lambda n: n > 1, n_qids_party)
len(list(filtered))


filter the data and keep only the qid of republicans and the democrats

In [ ]:
QID_republicans = "Q29468"
QID_democrats = "Q29552"
#df_wikidata_filtered = df_wikidata[(df_wikidata["party"][:, 0] == QID_democrats) | (df_wikidata["party"][:, 0] == QID_republicans)]
df_wikidata_filtered = df_wikidata_parties[df_wikidata_parties.apply(lambda x: (QID_republicans in x['party']) or (QID_democrats in x['party']) , axis=1)]

In [ ]:
df_wikidata_filtered

check if we have more than party per person for the filtered data

In [ ]:
qid_array_party = df_wikidata_filtered['party'].values
# number of qids per speaker
n_qids_party = [len(list) for list in qid_array_party ]
filtered = filter(lambda n: n > 1, n_qids_party)
len(list(filtered))

In [ ]:
# Merge quotebank data with wikidata 
df_merged = df_filtered.merge(right=df_wikidata_filtered, how='inner', left_on='qids', right_on='id')

In [ ]:
# Drop the id column because we already have the qid
df_merged = df_merged.drop(labels='id', axis=1)


In [ ]:
df_merged

In [ ]:
# Dump the new dataframe to json
path_to_out = '/content/quotes-2020-repub-dem.json.bz2'
df_merged.to_json(path_or_buf=path_to_out, compression='bz2')


In [ ]:
# Load the new file as a dataframe
file_path = '/content/drive/MyDrive/Quotebank_Repub_Dem/quotes-2018-repub-dem.json.bz2'
df = pd.read_csv(file_path, compression='bz2')

In [ ]:
df

In [ ]:
# Add number of quotations per qid to dataframe (Caution: this column is already aggregated)
df['quotationCounts'] = df.groupby(['qids'])['quoteID'].transform("count")

# Visualize the number of quotations per speaker grouped by Democrats and Republicans
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

subplot = axes[0];
subplot.hist(df[df['party'].str[0] == 'Q29468']['quotationCounts'].values, bins = 50, color="lightsteelblue")
subplot.set_title("Quotations per Speaker (Republican)")
subplot.set_xlabel("Quotation Count")
subplot.set_ylabel("Number of Speakers")
subplot.set_xlim([0,45000])
subplot.set_ylim([0,80000])

subplot = axes[1];
subplot.hist(df[df['party'].str[0] == 'Q29552']['quotationCounts'].values, bins = 50, color="cornflowerblue")
subplot.set_title("Quotations per Speaker (Democrat)")
subplot.set_xlabel("Quotation Count")
subplot.set_ylabel("Number of Speakers")
subplot.set_xlim([0,45000])
subplot.set_ylim([0,80000])

In [ ]:
# Top Speakers of the Republican Party
df[df['party'].str[0] == 'Q29468'].groupby(['qids']).mean().sort_values(by='quotationCounts', ascending=False)

In [ ]:
# Top Speakers of the Democratic Party
df[df['party'].str[0] == 'Q29552'].groupby(['qids']).mean().sort_values(by='quotationCounts', ascending=False)